<a href="https://colab.research.google.com/github/imkindaprogrammermyself/Camouflage-XML-Editor/blob/master/parse_replay_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import imageio
import pickle
from PIL import Image, ImageDraw, ImageOps
import numpy as np

image_map: Image.Image = Image.open("/content/minimap.png", "r")
image_water: Image.Image = Image.open("/content/minimap_water.png", "r")
image_water.paste(image_map, mask=image_map)
image_water = ImageOps.flip(image_water)


w, h = image_water.size
s_w, s_h = w / 1400, h / 1400


def get_scaled_xy(a, b):
    a = float(a)
    b = float(b)
    return float(a) * s_w + w / 2, float(b) * s_h + h / 2


writer = imageio.get_writer('/content/result.mp4', fps=500)
with open("/content/timed_position.pkl", "rb") as f:
    positional_data = pickle.load(f)
    positional_data = sorted(positional_data, key=lambda k: k[0])

    grouped_pos_data = {}
    last_position = {}

    for t_pos in positional_data:
        time, aid, teamId, name, isAbuser, isAlive, health, x, y = t_pos
        time = round(time)
        if time not in grouped_pos_data:
            grouped_pos_data[time] = [(aid, teamId, name, isAbuser, isAlive, health, x, y)]
        else:
            grouped_pos_data[time].append((aid, teamId, name, isAbuser, isAlive, health, x, y))

    for k, gpd in grouped_pos_data.items():
        if k % 100 == 0:
          print(f"Processed... {k}")
        for pd in gpd:
            aid, teamId, name, isAbuser, isAlive, health, x, y = pd
            sx, sy = get_scaled_xy(x, y)
            sx2, sy2 = sx - 2, sy - 2
            if teamId == 0:
                ImageDraw.Draw(image_water).ellipse((sx2, sy2, sx, sy), fill="red")
            else:
                ImageDraw.Draw(image_water).ellipse((sx2, sy2, sx, sy), fill="blue")
            image_water_copy = ImageOps.flip(image_water)
            image_water_copy = image_water_copy.resize((768, 768), resample=Image.NEAREST)
            writer.append_data(np.asarray(image_water_copy))
    writer.close()
    print("Done.")


Processed... 0
Processed... 100
Processed... 200
Processed... 300
Processed... 400
Processed... 500
Processed... 600
Processed... 700
Processed... 800
Processed... 900
Processed... 1000
Processed... 1100
Done.


In [0]:
!rm -rf /content/position

In [0]:
!mkdir /content/position